In [58]:
import pandas as pd
df4=pd.read_csv('/content/drive/MyDrive/AI/df4.csv') #PUBG의 데이터를 불러왔다
print(df4.shape) #df의 행렬크기

(4291447, 32)


In [59]:
df4=df4.drop('Unnamed: 0',axis=1) #예전 index가 남아있어서 제거

In [60]:
from sklearn.model_selection import train_test_split
train,test=train_test_split(df4,train_size=0.8,random_state=2) #train test 분리

In [61]:
train,vali=train_test_split(train,train_size=0.8,random_state=2)#train vali 분리
train.shape,vali.shape,test.shape

((2746525, 31), (686632, 31), (858290, 31))

In [62]:
target='winPlacePerc'  #target feature 분리
y_train=train[target]
X_train=train.drop(target,axis=1)
y_vali=vali[target]
X_vali=vali.drop(target,axis=1)
y_test=test[target]
X_test=test.drop(target,axis=1)
X_train.shape,X_vali.shape,X_test.shape,y_train.shape,y_vali.shape,y_test.shape

((2746525, 30), (686632, 30), (858290, 30), (2746525,), (686632,), (858290,))

In [63]:
#일단지워봅시다!

In [64]:
features=['damageDealt', 'DBNOs', 'kills', 'matchDuration', 'numGroups', 'w/t',
       'r/t', 'ttdistgame','groupId']

In [65]:
X_train=X_train[features]
X_vali=X_vali[features]
X_test=X_test[features]

In [66]:
!pip install category_encoders
!pip install eli5

In [67]:
from category_encoders import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import eli5
from eli5.sklearn import PermutationImportance
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import f1_score
from scipy.stats import randint,uniform
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score

{'class__min_child_weight': 5, 'class__max_depth': 3}  
0.9150702407738683

In [68]:
y_train.value_counts(normalize=True)

0    0.894997
1    0.105003
Name: winPlacePerc, dtype: float64

In [69]:
from collections import Counter #model의 가중치 넣는법 
counter = Counter(y_train) 
spw = counter[0] / counter[1]#scale_pos_weight = total_negative_lights / total_pos
spw

8.523549462018842

hp.choice(label, options) — Returns one of the options, which should be a list or tuple.

hp.randint(label, upper) — Returns a random integer between the range [0, upper).

hp.uniform(label, low, high) — Returns a value uniformly between low and high.

hp.quniform(label, low, high, q) — Returns a value round(uniform(low, high) / q) * q, i.e it rounds the decimal values and returns an integer.

hp.normal(label, mean, std) — Returns a real value that’s normally-distributed with mean and standard deviation sigma.

In [70]:
ordenc=OrdinalEncoder()

In [71]:
X_train=ordenc.fit_transform(X_train)
X_vali=ordenc.transform(X_vali)

In [82]:
X_train['groupId']=X_train['groupId'].astype('float64')

In [83]:
X_train.dtypes

damageDealt      float64
DBNOs              int64
kills              int64
matchDuration      int64
numGroups          int64
w/t              float64
r/t              float64
ttdistgame       float64
groupId          float64
dtype: object

In [77]:
X_vali.dtypes

damageDealt      float64
DBNOs              int64
kills              int64
matchDuration      int64
numGroups          int64
w/t              float64
r/t              float64
ttdistgame       float64
groupId          float64
dtype: object

In [72]:
space={'max_depth': hp.quniform("max_depth", 3, 18, 1), #마지막 옵션으로 나눈후 정수 반환
        'gamma': hp.uniform ('gamma', 1,9), #사이의 실수값 반환
        'reg_alpha' : hp.quniform('reg_alpha', 40,180,1),
        'reg_lambda' : hp.uniform('reg_lambda', 0,1),
        'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
        'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
        'n_estimators': 1500,
        'seed': 0
    }

In [73]:
def objective(space):
    clf=XGBClassifier(
                    n_estimators =space['n_estimators'], max_depth = int(space['max_depth']), gamma = space['gamma'],
                    reg_alpha = int(space['reg_alpha']),reg_lambda = int(space['reg_alpha']),min_child_weight=int(space['min_child_weight']),
                    colsample_bytree=int(space['colsample_bytree']),base_score=0.894997,scale_pos_weight=spw)
    #실수가들어가야하는값은 int변환
    eval = [( X_train, y_train), ( X_vali, y_vali)]
    
    clf.fit(X_train, y_train,
            eval_set=eval, eval_metric="auc",
            early_stopping_rounds=50,verbose=False)
    

    pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, pred>0.5)
    print ("SCORE:", accuracy)    
    return {'loss': -accuracy, 'status': STATUS_OK }

In [84]:
trials = Trials()

best_hyperparams = fmin(fn = objective,
                        space = space,
                        algo = tpe.suggest,
                        max_evals = 50,
                        trials = trials)

  0%|          | 0/50 [47:42<?, ?it/s, best loss: ?]


ValueError: ignored

In [ ]:
print("The best hyperparameters are : ","\n")
print(best_hyperparams)

'class__colsample_bytree': 0.9026379231331474,  
'class__gamma': 3,  
'class__learning_rate': 0.014516840274511014,  
  'class__max_depth': 4,  
   'class__min_child_weight': 1,  
      'class__n_estimators': 214,  
         'class__subsample': 0.7142428783542877  
train = 0.8488590862579588  
vali = 0.8485969692704555

In [ ]:
trials = Trials()

best_hyperparams = fmin(fn = objective,
                        space = space,
                        algo = tpe.suggest,
                        max_evals = 100,
                        trials = trials)

In [ ]:
print("The best hyperparameters are : ","\n")
print(best_hyperparams)

In [ ]:
y_train_pro=clf.predict_proba(X_train)[:,1]
 # proba가 [0의확률,1의확률] 두값이 존재 현재 필요한건 1의값

In [ ]:
def rocauc(y,y_pre_pro):
    fpr, tpr, thresholds = roc_curve(y, y_pre) #fpr tpr 임계점
    roc = pd.DataFrame({ #시각화를 위한 데이터프레임
        'FPR(Fall-out)': fpr, 
        'TPRate(Recall)': tpr, 
        'Threshold': thresholds
    })
    plt.scatter(fpr, tpr) #점찍음
    plt.title('ROC curve')
    plt.xlabel('FPR(Fall-out)')
    plt.ylabel('TPR(Recall)')

    optimal_idx = np.argmax(tpr - fpr) #tpr-fpr이 최대치가 되는부분을 찾음
    optimal_threshold = thresholds[optimal_idx] #고것이 추천 임계점

    print('idx:', optimal_idx, ', threshold:', optimal_threshold)
    y_pred_optimal = y_pre >= optimal_threshold #임계점보다 높은 값은 True
    print(classification_report(y, y_pred_optimal)) #classification을 보기위해 %=>true false로 바꾼것
    auc_score = roc_auc_score(y, y_pre)
    print('auc값:'auc_score)

In [ ]:
rocauc(y_vali,y_vali_pro)

In [ ]:
#여기서 한가지 의심드는게 GroupsId가 같으면 똥을싸도 캐리어의 점수를 따라가게된다

In [ ]:
def groupmax(model,X_):
    y_prepro=model.predict_proba(X_)[:,1] #모델의 예측값의 %
    y_prepro=pd.DataFrame(y_prepro)#df로 바꿔줌
    X_ts=X_.reset_index(drop=True)#prepro에 원래 index값이 안남아있어서 합치기위해 리셋인덱스
    X_ts=pd.concat([X_ts['groupId'],y_prepro],axis=1) #groupId와 y_prepro 합침
    X_ts=X_ts.sort_values(by='groupId').reset_index()#groupId순으로 정렬하고 기존 인덱스를 살림
    X_gr=X_ts.groupby('groupId')[0].max().reset_index()#groupId별 예측값의 최대값 groupid 살림
    X_max=pd.merge(X_ts,X_gr,how='left',on='groupId')#합치면서 각 groupId별로 max예측값을 갖게됨
    X_max=X_max.set_index('index').sort_index()#기본에 빼놨던 기본index를 기준으로 정렬
    y_max=X_max['0_y']#예측값만 꺼냄
    return print('y_max:',y_max)


In [ ]:
groupmax(clf,X_vali)

In [ ]:
rocauc(y_vali,y_max)


In [ ]:
xgbfi=randomcv.best_estimator_.named_steps['class'] #randomcv 안에있는 named_steps으로 xgb를 꺼냄

In [ ]:
importances = pd.Series(xgbfi.feature_importances_, X_train.columns) #xgb자체 feature importances
plt.figure(figsize=(10,10))
importances.sort_values().plot.barh();

In [ ]:
permuter=PermutationImportance(
    clf,
    scoring='f1_weighted',
    n_iter=5,
    random_state=2)


In [ ]:
permuter.fit(X_vali,y_vali)

In [ ]:
feature_names = X_train.columns.tolist()
pd.Series(permuter.feature_importances_, feature_names).sort_values()

In [ ]:
eli5.show_weights(
    permuter, 
    top=None, 
    feature_names=feature_names 
) #xgb기본 feature importances와는 다를걸 예상했지만 이건 달라도 너무달라서 충격적이다 
# ttdistgame는 개인의 이동거리/각 게임당 전인원의 이동거리로 만든것인데 너무 할 정도로 높다...
# 전처리중에 아예 타겟값을 베이스로 만든 피쳐는 없었음.. 

기존결과  
'class__colsample_bytree': 0.9026379231331474,  
'class__gamma': 3,  
'class__learning_rate': 0.014516840274511014,  
'class__max_depth': 4,  
'class__min_child_weight': 1,  
'class__n_estimators': 214,  
'class__subsample': 0.7142428783542877  
train = 0.8488590862579588  
vali = 0.8485969692704555  